## Edge cases

* When the number of tests and associated points doesn't add up to the total points in the cell
* What happens when the student doesn't actually generate a plot -- 

### It would be cool to be able to capture and print out the specifics of the error 
* Capture the specifics of what went wrong-- and return it as a friendly error message
* currently this all fails if the student gets things wrong! 

https://github.com/earthlab/earth-analytics-lessons/blob/master/courses/earth-analytics-python/03-intro-to-python-and-time-series-data/2018-02-05-intro-to-python-time-series-data-landing-page.ipynb

## Things that we want to check

1. order of package imports + all imports at the top
2. all imports are USED in the notebook
2. notebook starts at `ln 1`
3. notebook RUNS (all cells run)
4. linting -- autopep8 applied. 


## Earth Analytics Homework - Use Time Series Data with Python
![](EarthlabSquare.png)


For your assignment be sure to

* Load all libraries at the top of your notebook (before your code begins) in their own notebook cell.
* Follow pep-8 code style guidelines. As you do this consider
        # Comment spacing and capitalization
        Write useful comments - not too many but enough to guide a reader through your workflow
        Use line spaces and returns in your code to make it easier to read where needed
* Label all plot x and y axes - include units as necessary
* Format dates on the x axis so they are readable
* Use pandas to process your data
* Be sure to cite the source of the data used for each plot. HINT: there are metadata documents in the data folder that will help you with this.

For each plot be sure to add a markdown cell below with 2-4 sentences that plot that explains what this plot shows.

# This is an actual rubric from my class. 
i'll try to download it into a text format but it's silly html right now from the LMS
![](rubric-1.png)
![](rubric-2.png)
![](rubric-3.png)

In [28]:
# Do not touch this cell. Please place the imports required from your homework in the cell below
# <Can we lock cells?>
% timeit
import matplotcheck.notebook as nb
import matplotcheck.timeseries as ts
import matplotcheck.raster as ra

## Grading to do
check the cell below for (in order of importance)
1. imports not used in the notebook
2. order of imports folowing PEP guidelines (most common furst to third party last)
3. spell check?

In [2]:
# Add the required packages needed to run your code here
# We normally check to see if these on in the correct order following PEP guidelines (most common first)
# BEGIN SOLUTION
import os
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
import seaborn as sns
import numpy as np
import pandas as pd
import earthpy as et
import hydrofunctions as hf
import urllib

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

# prettier plotting with seaborn
sns.set(font_scale=1.5)
sns.set_style("whitegrid")

# END SOLUTION

In [3]:
# Get data
data = et.data.get_data("colorado-flood")
os.chdir(os.path.join(et.io.HOME, 'earth-analytics'))

## Homework PLOT 1 - Time series / pandas

Use pandas to open and process the `data/colorado-flood/discharge/06730200-discharge-daily-1986-2013.txt` txt file. Using this data, create a plot of monthly maximum stream discharge from 1990-2014. Document your steps used to process the data in a markdown cell BELOW the plot use pandas, make sure the dataframe has a datetime index calculate a monthly max value

Subset the data to the time period: `1990-2014` using pandas syntax `["":""]`. This means that your date column should be setup as an index.

In [18]:
# BEGIN SOLUTION

# import file
f = "data/colorado-flood/discharge/06730200-discharge-daily-1986-2013.txt"
discharge = pd.read_csv(f,
                        skiprows=23,
                        header=[1, 2],
                        sep='\t',
                        parse_dates=[2])
# drop one level of index
discharge.columns = discharge.columns.droplevel(1)
# set the date column as the index
discharge = discharge.set_index(["datetime"])

monthly_max_all = discharge.resample("M").max()
monthly_max = monthly_max_all['1990':'2014']

fig, ax = plt.subplots(figsize=(10, 8))
ax.scatter(x=monthly_max.index,
           y=monthly_max["17663_00060_00003"],
           color="purple")
ax.set_title(
    "HW Plot 1: Stream Discharge - Monthly Max Value\n be sure to add x and y labels (not shown here)")

ax.set(xlabel="Date")

# I believe we aren't supposed to use plt.show()
#plt.show()

# END SOLUTION

### DO NOT REMOVE LINE BELOW ###
ts_1_plot = nb.convert_axes(plt)

In [24]:
# not sure how to do this. also i remember yuvi having a way to do this that was language agnostic.
def run_test(test, total_points, correct_message, error_message):
    # test each input

    single_result = {"test_desc": 'Plot Structure', "points": 0}
    try:
        test
        single_result["points"] = total_points
        single_result["message"] = correct_message
    except AssertionError as e:
        single_result["message"] = error_message
        single_result["error_specifics"] = e.message
        
    return single_result
    
def run_all_tests(list_of_tests):
    total_points = 0
    all_results = []
    for a_test in list_of_tests:
        result = run_test(a_test[0],a_test[1],a_test[2],a_test[3]) 
        points = result["points"]
        total_points += points
        all_results.append(result)
    
    # Print out tests - this could be a little print helper too?
    for r in all_results:
        #total_points += float(r['points'])
        print("Testing {}. {} Specific Error: {} (points = {})".format(
            r['test_desc'], r['message'], r['error_specifics'], r['points']))
    print("You recieved ", total_points, " points for this set of tests")
    return total_points

In [25]:
# Locked Grading Cell - Homework plot 1 -
hw_plot1 = ts.TimeSeriesTester(ts_1_plot)

# Public tests
plot_type_test = [hw_plot1.assert_plot_type("line"), 1,
                  "Plot type Pass! You plot is a scatter plot!",
                  "Ooops, please check to ensure your plot is a scatter plot."]

plot_title_test = [hw_plot1.assert_title_contains(["Stream discharge"]), 1,
                   "Plot title contains the words `Stream discharge`",
                   "Oops, please make sure your plot title words"]
plot_xticks_date_test = [hw_plot1.assert_xticks_reformatted(
    tick_size="large", loc_exp="year", m="Please fix this"), 1,
    "X ticks pass! dates are formatted as years",
    "X ticks fail! Dates should be formated using year"]
plot_x_lab_test = [hw_plot1.assert_axis_label_contains(axis="x", lst=["Date"]),
                   1,
                   "X label test pass! X axis contains the word `Date`",
                   "X label test fail! X axis should contain a lable with the word `Date` in it"]

run_all_tests([plot_type_test,
               plot_title_test,
               plot_xticks_date_test,
               plot_x_lab_test])

AssertionError: Plot is not of type line

In [7]:
### BEGIN HIDDEN TESTS
# Add test to ensure the df has the correct values

data_test = [hw_plot1.assert_xydata(xy_expected=monthly_max.reset_index(),
                                    xtime=True,
                                    xcol="datetime",
                                    ycol="17663_00060_00003"), 6,
             "Testing plot data: Your data look correct",
             "Testing plot data: Ooops the data in your plot don't look right"]
run_all_tests([data_test])

### END HIDDEN TESTS

# Note i think i probably need to update MPL and did submit an issue about the warning 
# i see below. i think i understand what to fix!

Testing Plot Structure. Testing plot data: Your data look correct (points = 6)
You recieved  6  points for this set of tests


/anaconda3/envs/earth-analytics-python/lib/python3.6/site-packages/numpy/core/numeric.py:2586: FutureWarning: Converting timezone-aware DatetimeArray to timezone-naive ndarray with 'datetime64[ns]' dtype. In the future, this will return an ndarray with 'object' dtype where each element is a 'pandas.Timestamp' with the correct 'tz'.
	To accept the future behavior, pass 'dtype=object'.
	To keep the old behavior, pass 'dtype="datetime64[ns]"'.
  a1, a2 = asarray(a1), asarray(a2)


6

In [8]:
# Tests not using wrappers
points = 10

# Locked Grading Cell - Homework plot 1 -
hw_plot1 = ts.TimeSeriesTester(ts_1_plot)

# Plot type and title tests (5)
if hw_plot1.assert_plot_type("scatter"):
    points += 1
if hw_plot1.assert_title_contains(["Stream discharge"]):
    points += 1


# Plot axis tests
if hw_plot1.assert_xticks_reformatted(tick_size="large", loc_exp="year", m="Please fix this"):
    points += 1

if hw_plot1.assert_axis_label_contains(axis="x", lst=["Date"]):
    points += 1

# BEGIN HIDDEN TESTS
# Add test to ensure the df has the correct values
if hw_plot1.assert_xydata(xy_expected=monthly_max.reset_index(),
                          xtime=True,
                          xcol="datetime",
                          ycol="17663_00060_00003",
                          m="oops - you messed up"):
    points += 6

# END HIDDEN TESTS
points

/anaconda3/envs/earth-analytics-python/lib/python3.6/site-packages/numpy/core/numeric.py:2586: FutureWarning: Converting timezone-aware DatetimeArray to timezone-naive ndarray with 'datetime64[ns]' dtype. In the future, this will return an ndarray with 'object' dtype where each element is a 'pandas.Timestamp' with the correct 'tz'.
	To accept the future behavior, pass 'dtype=object'.
	To keep the old behavior, pass 'dtype="datetime64[ns]"'.
  a1, a2 = asarray(a1), asarray(a2)


10

In [9]:
# testing wrapper function
# this was karen's original take on the testing! note that all tests pass in this notebook so 
# havent tried to break things yet

def run_tests(plot):
    results = []
    total_points = 0

    single_result = {"test_desc": 'Are x-axis ticks reformatted?', "points": 0}
    try:
        hw_plot1.assert_xticks_reformatted(
            tick_size="large", loc_exp="year", m="Please fix this")
        single_result["points"] = 1
        single_result["message"] = "Looks good!"
    except AssertionError:
        single_result["message"] = "x ticks have not been reformatted properly"
    results.append(single_result)

    for r in results:
        total_points += float(r['points'])
        print("Testing {}. Result:  {} (points = {})".format(
            r['test_desc'], r['message'], r['points']))

    return total_points


hw_plot1 = ts.TimeSeriesTester(ts_1_plot)
run_tests(hw_plot1)

Testing Are x-axis ticks reformatted?. Result:  Looks good! (points = 1)


1.0

If all tests pass above it would be nice to be able to provide a custom message that says -- all looks good... 

# Lets try this with spatial data - new homework in same notebook

In [10]:
# Import your packages and download data / set working directory here
# BEGIN SOLUTION
import os
from glob import glob
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np

import rasterio as rio
from rasterio.plot import plotting_extent
from rasterio.mask import mask
import geopandas as gpd

from shapely.geometry import mapping, box
# to create geojson object for raster cropping
from geojson import Polygon

import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep

# Get data and set working dir
data1 = et.data.get_data('cs-test-naip')
data2 = et.data.get_data('cold-springs-fire')
# set working directory to your home dir/earth-analytics
os.chdir(os.path.join(et.io.HOME, 'earth-analytics'))
# END SOLUTION

In [11]:
# BEGIN SOLUTION
# Open fire boundary
fire_bound_path = "data/cold-springs-fire/vector_layers/fire-boundary-geomac/co_cold_springs_20160711_2200_dd83.shp"
fire_bound = gpd.read_file(fire_bound_path)

# Open several spatial datasets

# Open  NAIP 2015 data & reproject fire boundary
with rio.open("data/cold-springs-fire/naip/m_3910505_nw_13_1_20150919/crop/m_3910505_nw_13_1_20150919_crop.tif") as src:
    #naip_2015_crop, naip_2015_affine = mask(src, [naip_clip_extent], crop=True)
    naip_2015_crop = src.read()
    naip_2015_bds = src.bounds
    fire_bound_utmz13 = fire_bound.to_crs(src.crs)
    naip_2015_ext = plotting_extent(naip_2015_crop[0], src.transform)
    meta = src.profile

# Create a geojson extent box to crop the other data
naip_clip_extent = mapping(box(*naip_2015_bds))

# Crop 2017 NAIP data to the 2015 data boundary
with rio.open("data/cs-test-naip/m_3910505_nw_13_1_20170902.tif") as src:
    naip_2017_crop, naip_2017_affine = es.crop_image(
        src, [box(*naip_2015_bds)])
    naip_2017_crop = naip_2017_crop.astype(int)

# END SOLUTION

In [17]:
# Plot the data - place your final data here

# BEGIN SOLUTION

# Add plot code only for plots 1 and 2 here
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 10))
ep.plot_rgb(naip_2017_crop,
            rgb=[0, 1, 2],
            extent=naip_2015_ext,
            title="Homework PLOT 1: NAIP 2017 Post Fire RGB Image\n Cropped",
            ax=ax1)
fire_bound_utmz13.plot(ax=ax1, color='None',
                       edgecolor='white', linewidth=2)


# plot 2017 cropped data
ep.plot_rgb(naip_2017_crop,
            rgb=[3, 0, 1],
            extent=naip_2015_ext,
            title="Homework PLOT 2: NAIP 2017 Post Fire CIR Image",
            ax=ax2)
fire_bound_utmz13.plot(ax=ax2, color='None',
                       edgecolor='white', linewidth=2)
plt.show()
# END SOLUTION

### DO NOT REMOVE LINE BELOW ###
raster_plot = nb.convert_axes(plt, which_axes="all")

/anaconda3/envs/earth-analytics-python/lib/python3.6/site-packages/matplotlib/figure.py:445: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  % get_backend())
/anaconda3/envs/earth-analytics-python/lib/python3.6/site-packages/matplotlib/figure.py:445: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  % get_backend())
/anaconda3/envs/earth-analytics-python/lib/python3.6/site-packages/matplotlib/figure.py:445: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  % get_backend())


In [13]:
# Here i have to create a set of tests for a plot with subplots. this might involve loops
hw_raster_plot1 = ra.RasterTester(raster_plot[0])
hw_raster_plot2 = ra.RasterTester(raster_plot[1])

test_axis_off_plot1 = [hw_raster_plot1.assert_axis_off(),
                       1,
                       "RGB Image: Axis format looks great!",
                       "RGB Image: Looks like your axis is still on. Please turn axis_off."]

test_axis_off_plot2 = [hw_raster_plot2.assert_axis_off(),
                       1,
                       "CIR Image: Axis format looks great!",
                       "CIR Image: Looks like your axis is still on. Please turn axis_off."]

# Check plot extent ?? -- not sure i can do that yet with MPC functionality but it would be easy to add


# Check title
test_title_plot1 = [hw_raster_plot1.assert_title_contains(["NAIP", "2017", "RGB"]),
                    2,
                    "Title looks good",
                    "oops check your title"]
test_title_plot2 = [hw_raster_plot2.assert_title_contains(["NAIP", "2017", "CIR"]),
                    2,
                    "Title looks good",
                    "oops check your title"]


run_all_tests([test_axis_off_plot1, 
               test_axis_off_plot2, 
               test_title_plot1, 
               test_title_plot2])

Testing Plot Structure. RGB Image: Axis format looks great! (points = 1)
Testing Plot Structure. CIR Image: Axis format looks great! (points = 1)
Testing Plot Structure. Title looks good (points = 2)
Testing Plot Structure. Title looks good (points = 2)
You recieved  6  points for this set of tests


6

In [14]:
# Hidden raster tests
# in this case our plot has several axes which we need to test individually.
# so the use case here will require loops which a function will be good for. writing it all out first.

# HIDDEN TESTS
# Calculate expected raster data - plot one RGB image
rgb = [0, 1, 2]
rgb_bands = naip_2017_crop[rgb, :, :]
im = es.bytescale(rgb_bands).transpose([1, 2, 0])

# CIR image - plot 2
hw_raster_plot2 = ra.RasterTester(raster_plot[1])

rgb2 = [3, 0, 1]
rgb_bands2 = naip_2017_crop[rgb2, :, :]
im2 = es.bytescale(rgb_bands2).transpose([1, 2, 0])

data_test_1 = [hw_raster_plot1.assert_image(im), 5,
              "RGB Image Data Test: RGB data look correct!",
              "wrong data"]
data_test_2 = [hw_raster_plot2.assert_image(im2), 5,
              "CIR Image Data Test: CIR data look correct!",
              "wrong data"]
## test that the fire boundary is correct
# Test that each plot has the fire boundary overlaid on top of it
# we will need to test what happens if the extent of the plot isn't right...
# I added this to MPC by having the raster test inheret from the vector testor. it's a PR currently.

# Convert to vertices that can be compared
fire_bound_exp = hw_raster_plot2._convert_multipolygons(fire_bound_utmz13.geometry)

test_fire_bound_plot1 = [hw_raster_plot1.assert_polygons(fire_bound_exp),
                        5,
                        "Correct message",
                        "You got it wrong"]
test_fire_bound_plot2 = [hw_raster_plot2.assert_polygons(fire_bound_exp),
                        5,
                        "Correct message",
                        "You got it wrong"]

run_all_tests([data_test_1, data_test_2, test_fire_bound_plot1, test_fire_bound_plot2])

Testing Plot Structure. RGB Image Data Test: RGB data look correct! (points = 5)
Testing Plot Structure. CIR Image Data Test: CIR data look correct! (points = 5)
Testing Plot Structure. Correct message (points = 5)
Testing Plot Structure. Correct message (points = 5)
You recieved  20  points for this set of tests


20